In [ ]:
import os
import glob

import numpy as np
import pandas as pd
import xarray as xr

from metpy.units import units

import cartopy.crs as ccrs

import bokeh.palettes
from bokeh.models import FixedTicker

import holoviews as hv
from holoviews import opts
import geoviews.feature as gf

from uviz.plotting.hv_plot import Polymesh, plot_native
from uviz.plotting.utils import basin_bboxes, CustomColorbars
from uviz.plotting.processing import ModelData

from uviz.datashader_tools.visualization import diverging_colormap

hv.extension("bokeh")

In [ ]:
tracks_dir = r"/gpfs/group/cmz5202/default/cnd5285/synth_events"
track_folders = glob.glob(os.path.join(tracks_dir, '*storm*'))
parent_folders = [os.path.join(folder, '28km') for folder in track_folders]
child_folders = [os.path.join(folder, '3km') for folder in track_folders]

In [ ]:
%%time
all_storms = ModelData(tracks_dir, regridded=False).storms

# It's actually faster to read in data non-parallel (1 min 11s) vs parallel (2 min 54s)

In [ ]:
storm_names = [x for x in all_storms.keys() if x != 'Charley']

In [ ]:
def return_polymesh(storm, model, var, nan_idx=None, dims={}):
    """
    Function to automate Polymesh ds retrieval
    """
    if var in ['PRECT_TOT', 'PRECT_MAX', 'PRECT_MAX_CONV']:
        if model == 'cam':
            ds = 'h4pn_ds'
            mesh = 'parent_grid'
        elif model == 'mpas':
            ds = 'h4cn_ds'
            mesh = 'child_grid'
    else:
        if model == 'cam':
            ds = 'h3pn_ds'
            mesh = 'parent_grid'
        elif model == 'mpas':
            ds = 'h3cn_ds'
            mesh = 'child_grid'
            
    if var not in ['PRECT_TOT', 'PRECT_MAX', 'U10_MAX', 'WSP850_MAX', 'PRECT_MAX_CONV'] and not dims:
        raise ValueError('Must supply dims for time or level dependent variable.')
    
    # Makes copy of xr Dataset so it doesn't overwrite data in place with nans
    sel_ds = all_storms[storm][ds].copy()
    sel_mesh = all_storms[storm][mesh]
    
    # Masks non-target values with nans (for landmask/subset purposes)
    if isinstance(nan_idx, np.ndarray):
        sel_ds[var].loc[~sel_ds.ncol.isin(nan_idx)] = np.nan
    
    df = Polymesh(sel_mesh, sel_ds, model=model).data_mesh(target_var=var, dims=dims, fill="faces")
    
    return df

In [ ]:
def spatial_subset(ds, bbox, overland=False, get_idx=False, **kwargs):
    
    # Masks out land values if True
    if overland == True:
        ds = mask_land(ds)

    # Retrieves bounding box coordinates
    if isinstance(bbox, (dict, str)):
        lons, lats = basin_bboxes(bbox)
    else:
        lons, lats = bbox

    # Subsets data spatially (for regridded data)
    if 'lon' in ds.dims:
        ds = ds.sel(lon=slice(lons[0], lons[1]), lat=slice(lats[0], lats[1]))

    # For native data
    elif 'ncol' in ds.dims:
        # Retrieves subset with nans dropped
        ds = ds.where((ds['lon'] >= lons[0]) & (ds['lon'] <= lons[1]) & 
                      (ds['lat'] >= lats[0]) & (ds['lat'] <= lats[1]), drop=True)

    return ds

In [ ]:
def get_subset_idx(ds, bbox, overland=True):
    
    # Retrieves bounding box coordinates
    if isinstance(bbox, (dict, str)):
        lons, lats = basin_bboxes(bbox)
    else:
        lons, lats = bbox
        
    # Gets indices to feed to ncol (for plotting)
    if 'ncol' in ds.dims:
        # Retrieves subset but leaves nans in place
        sub_ds = ds.where((ds['lon'] >= lons[0]) & (ds['lon'] <= lons[1]) &
                          (ds['lat'] >= lats[0]) & (ds['lat'] <= lats[1]))
        
        # Retrieves latitude and longitude values
        sub_lons = sub_ds.lon.values
        sub_lats = sub_ds.lat.values
        
        # Gets ncol indices of non-nan values
        lon_idx = np.argwhere(~np.isnan(sub_lons)).flatten()
        lat_idx = np.argwhere(~np.isnan(sub_lats)).flatten()
        
        # Gets union of both indices
        sub_idx = np.intersect1d(lon_idx, lat_idx)
        
        if overland == True:
            # Masks out land values
            land_idx = mask_land(ds, return_idx=True)[1]
            
            # Also gets union of subset indices and landmask indices
            idx = np.intersect1d(sub_idx, land_idx)
        else:
            idx = sub_idx
                
        return idx
    
    # Add else clause here + error message for regridded data

In [ ]:
import datashader as ds
from holoviews import opts

def plot_overlays(base_df, contour_df, plot='shear'):
    # Defines defaults
    w = 1000
    h = 700
    
    proj = ccrs.PlateCarree()
    lon_range, lat_range = basin_bboxes('florida')
    x_range, y_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
    x_range = tuple(x_range)
    y_range = tuple(y_range)
    
    plot_opts = dict(width=w, height=h, xaxis=None, yaxis=None)
    
    lw = 2.0
    coastline_opts = dict(scale='10m', line_color='#000000', line_width=lw, width=w, height=h)
    states_opts = dict(scale='50m', line_color='#000000', line_width=lw, width=w, height=h, fill_color='none')
    
    if plot == 'shear':
        base_cmap = 'gist_yarg'
        base_levs = CustomColorbars(target_cmap='FLUT').out_levels
        base_cmin = base_levs[0]
        base_cmax = base_levs[-1]
        base_cmap_opts = dict(cmap=base_cmap, clim=(base_cmin, base_cmax), colorbar=False)
        base_plot_opts = dict(**plot_opts, **base_cmap_opts)
        
        contour_cmap = ["#00ff00", "#00ff00", "#fffe00", "#ff0200"]
        contour_levs = [0.0, 10.0, 20.0, 30.0]
        
    elif plot == 'cape':
        base_cmap = ['#ffffff', '#ffffff', "#4bd2f7", "#699fd0", "#3c4bac", 
                     "#3cf74b", "#3cb447", "#3c8743", "#1f4723", "#f7f73c"]
        base_levs = CustomColorbars('nws_precip').out_levels[:11]
        base_cmin = base_levs[0]
        base_cmax = base_levs[-1]
        base_cmap_opts = dict(cmap=base_cmap, clim=(base_cmin, base_cmax), colorbar=False)
        base_plot_opts = dict(**plot_opts, **base_cmap_opts, alpha=0.75, color_levels=base_levs)
         
        contour_cmap = ["#cc0205", "#ad0006", "#8f0004", "#720000"]
        contour_levs = [500, 1000, 1500, 2000]
        
    contour_cmin = contour_levs[0]
    contour_cmax = contour_levs[-1]
    contour_cmap_opts = dict(cmap=contour_cmap, clim=(contour_cmin, contour_cmax), colorbar=False)
    contour_plot_opts = dict(**plot_opts, **contour_cmap_opts)
    
    coastline_layer = gf.coastline(projection=proj).opts(**coastline_opts, xlim=x_range, ylim=y_range)
    states_layer = gf.states(projection=proj).opts(**states_opts, xlim=x_range, ylim=y_range)

    # Sets up basemap
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)

    # Adds base layer polygons
    agg_base = cvs.polygons(base_df, geometry='geometry', agg=ds.mean('faces'))

    # Rasterizes layer
    hv_img_base = hv.element.raster.Image(agg_base).opts(**base_plot_opts)

    # Adds geographic layers
    hv_img_base = hv_img_base * coastline_layer * states_layer

    # Adds shear polygons
    agg_contour = cvs.polygons(contour_df, geometry='geometry', agg=ds.mean('faces'))

    # Rasterizes layer
    hv_img_contour = hv.element.raster.Image(agg_contour).opts(**contour_plot_opts)

    # Retrieves contours from shear layer and adds to plot
    out_plot = hv_img_base * hv.operation.contours(hv_img_contour, levels=contour_levs)
    out_plot = out_plot.opts(opts.Contours(**contour_cmap_opts, line_width=lw+3.0, show_legend=False))
    out_plot = out_plot.opts(**plot_opts)
    
    return out_plot

# OLR Figs

In [ ]:
# Subsets data by user-defined bounding box
storm_names = [x for x in all_storms.keys() if x not in ['Irma', 'Ian', 'Isaac', 'Charley', 'Fay']]

# Just for Charley, Irma, and Ian
#storm_names = [x for x in all_storms.keys() if x in ['Irma', 'Ian', 'Isaac' 'Fay']]

In [ ]:
%%time
h3pn_subsets = list(map(lambda x: spatial_subset(all_storms[x]['h3pn_ds'], 'florida'), storm_names))
h3cn_subsets = list(map(lambda x: spatial_subset(all_storms[x]['h3cn_ds'], 'florida'), storm_names))

# Finds the timestep at which the maximum OLR occurs
delay = 5 # trying to avoid model spin-up
clip = -10 # trying to catch peak within middle of model run
min_slps_h3pn = [ds['PSL'].isel(time=slice(delay, clip)).compute().min('ncol').argmin(dim='time').values+delay for ds in h3pn_subsets]
min_slps_h3cn = [ds['PSL'].isel(time=slice(delay, clip)).compute().min('ncol').argmin(dim='time').values+delay for ds in h3cn_subsets]

In [ ]:
min_slps_h3cn

In [ ]:
min_slps_h3pn

In [ ]:
# Use max intensity parent timesteps for child
min_slps_h3pn = [15, 18, 11, 14, 39, 26, 17, 7, 12]

p_olr_dfs = [return_polymesh(s, 'cam', 'FLUT', dims=dict(time=i)) for s, i in zip(storm_names, min_slps_h3pn)]
c_olr_dfs = [return_polymesh(s, 'mpas', 'FLUT', dims=dict(time=i)) for s, i in zip(storm_names, min_slps_h3pn)]
time_str_c = [pd.to_datetime(str(all_storms[s]['h3cn_ds'].isel(time=t).time.values)).strftime('%m%d%Y%H') for s, t in zip(storm_names, min_slps_h3pn)]
time_str_p = [pd.to_datetime(str(all_storms[s]['h3pn_ds'].isel(time=t).time.values)).strftime('%m%d%Y%H') for s, t in zip(storm_names, min_slps_h3pn)]

In [ ]:
%%time
p_olr_dfs = [return_polymesh(s, 'cam', 'FLUT', dims=dict(time=i)) for s, i in zip(storm_names, min_slps_h3pn)]
c_olr_dfs = [return_polymesh(s, 'mpas', 'FLUT', dims=dict(time=i)) for s, i in zip(storm_names, min_slps_h3cn)]

In [ ]:
# Converts timesteps to strings
time_str_p = [pd.to_datetime(str(all_storms[s]['h3pn_ds'].isel(time=t).time.values)).strftime('%m%d%Y%H') for s, t in zip(storm_names, min_slps_h3pn)]
time_str_c = [pd.to_datetime(str(all_storms[s]['h3cn_ds'].isel(time=t).time.values)).strftime('%m%d%Y%H') for s, t in zip(storm_names, min_slps_h3cn)]

In [ ]:
p_shear_dfs = [return_polymesh(s, 'cam', 'SHEAR_TC', dims=dict(time=i)) for s, i in zip(storm_names, min_slps_h3pn)]
c_shear_dfs = [return_polymesh(s, 'mpas', 'SHEAR_TC', dims=dict(time=i)) for s, i in zip(storm_names, min_slps_h3pn)]

In [ ]:
for df in c_shear_dfs:
    print(df['faces'].max())

In [ ]:
for df in c_shear_dfs:
    print(df['faces'].min())

In [ ]:
cape_color = "#cc0205" # levels = 500, 1000, 1500, 2000
cin_line_color = "#0719fb"

hv.Polygons(polymesh_df, vdims=['faces']).opts(line_color='faces', fill_color='None', clim=(0, 100)) 

## Including shear

In [ ]:
cam_shear_plots = [plot_overlays(x, y, 'shear') for x, y in zip(p_olr_dfs, p_shear_dfs)]
mpas_shear_plots = [plot_overlays(x, y, 'shear') for x, y in zip(c_olr_dfs, c_shear_dfs)]

In [ ]:
[hv.save(fig, f'../figs/new_tracks/{storm}/cam_shear_olr_{ts}.png', dpi=300, center=False) for fig, storm, ts in zip(cam_shear_plots, storm_names, time_str_p)]

In [ ]:
[hv.save(fig, f'../figs/new_tracks/{storm}/mpas_shear_olr_{ts}.png', dpi=300, center=False) for fig, storm, ts in zip(mpas_shear_plots, storm_names, time_str_p)]

In [ ]:
import matplotlib as mpl
import matplotlib.colors as mcolors

# Defines width and height for plot
w=2000
h=1400
#h=1600

w = int(w/2)
h = int(h/2)

# Defines rasterization options (expose options with `hv.help(hds_rasterize)`)
datashader_opts = dict(aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=10)

# Defines colorbar/colormap options
flut_cmap = 'gist_yarg' # mpl.colormaps['gist_yarg']
flut_levs = CustomColorbars(target_cmap='FLUT').out_levels
    
cmin = flut_levs[0]
cmax = flut_levs[-1]
clabel = r'Upwelling Longwave Flux [W/m^2]'
cbar_opts = dict(height=40, border_line_width=3)
cmap_opts = dict(cmap=flut_cmap, clim=(cmin, cmax), colorbar=False, 
                 colorbar_position='bottom', clabel=clabel)

# Defines font options
# font_opts = dict(fontsize=dict(title='35pt', clabel='27pt', cticks='25pt'))
font_opts = dict(fontsize=dict(title='35pt', clabel='50pt', cticks='40pt'))
if h < 1600:
    cmap_opts['colorbar'] = False
elif h == 1600:
    cmap_opts['colorbar'] = True

# Defines plotting options (expose options with `hv.help(hv.Polygons)`)
olr_plot_opts = dict(width=w, height=h, xaxis=None, yaxis=None,
                 colorbar_opts=cbar_opts, **cmap_opts, **font_opts)

# Defines coastline plotting options
#coastline_opts = dict(scale='10m', line_color='#f8f8f8', line_width=2.0, width=w, height=h)
lw = 2.0
coastline_opts = dict(scale='10m', line_color='#FFFFFF', line_width=lw, width=w, height=h)
states_opts = dict(scale='50m', line_color='#FFFFFF', line_width=lw, width=w, height=h, fill_color='none')

# Defines bounding bbox
bbox = basin_bboxes('florida')

# Packs into one dict for cleaner code
plotting_kwargs = {'raster':True, 'plot_bbox':bbox, 'holoviews_kw':plot_opts, 'datashader_kw':datashader_opts, 
                   'coastline_kw':coastline_opts, 'states_kw':states_opts}

#cam_olr_test = plot_native(p_olr_dfs[1], **plotting_kwargs)

In [ ]:
proj = ccrs.PlateCarree()
lon_range, lat_range = basin_bboxes('florida')
x_range, y_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
x_range = tuple(x_range)
y_range = tuple(y_range)

w=1000
h=700

# Defines colorbar/colormap options
shear_cmap = CustomColorbars(target_cmap='shear_cimss').out_cmap
#shear_cmap = ["#00ff00", "#00ff00", "#fffe00", *[*["#ff0200"]*8]]
shear_cmap = ["#00ff00", "#00ff00", "#fffe00", "#ff0200"]
shear_levs = [0.0, 10.0, 20.0, 25.0]
#shear_levs = CustomColorbars(target_cmap='shear_cimss').out_levels
shear_cmap_opts = dict(cmap=shear_cmap, clim=(shear_levs[0], shear_levs[-1]), colorbar=False)
shear_plot_opts = dict(width=w, height=h, xaxis=None, yaxis=None, **shear_cmap_opts)

plot_opts = dict(width=w, height=h, xaxis=None, yaxis=None)

lw = 2.0
coastline_opts = dict(scale='10m', line_color='#FFFFFF', line_width=lw, width=w, height=h)
states_opts = dict(scale='50m', line_color='#FFFFFF', line_width=lw, width=w, height=h, fill_color='none')

coastline_layer = gf.coastline(projection=proj).opts(**coastline_opts, xlim=x_range, ylim=y_range)
states_layer = gf.states(projection=proj).opts(**states_opts, xlim=x_range, ylim=y_range)

In [ ]:
import datashader as ds
from holoviews import opts

# Sets up basemap
cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)

# Adds OLR polygons
agg_olr = cvs.polygons(c_olr_dfs[1], geometry='geometry', agg=ds.mean('faces'))

# Rasterizes layer
hv_img_olr = hv.element.raster.Image(agg_olr).opts(**olr_plot_opts)

# Adds geographic layers
hv_img_olr = hv_img_olr * coastline_layer * states_layer

# Adds shear polygons
agg_shear = cvs.polygons(c_shear_dfs[1], geometry='geometry', agg=ds.mean('faces'))

# Rasterizes layer
hv_img_shear = hv.element.raster.Image(agg_shear).opts(**shear_plot_opts)

# Retrieves contours from shear layer and adds to plot
shear_contours = hv_img_olr * hv.operation.contours(hv_img_shear, levels=shear_levs)
shear_contours = shear_contours.opts(opts.Contours(**shear_cmap_opts, line_width=2.0, show_legend=False))
shear_contours = shear_contours.opts(**plot_opts)

In [ ]:
%%time
hv.save(shear_contours, f'../figs/new_tracks/{storm_names[1]}/mpas_olr_shear_{time_str_p[1]}.png', dpi=300, center=False)

## Testing

In [ ]:
%%time
# Finds the timestep at which the maximum OLR occurs
delay = 5 # trying to avoid model spin-up
clip = -10 # trying to catch peak within middle of model run
max_fluts_h3pn = [ds['FLUT'].isel(time=slice(delay, clip)).compute().min('ncol').argmin(dim='time').values for ds in h3pn_subsets]
max_fluts_h3cn = [ds['FLUT'].isel(time=slice(delay, clip)).compute().min('ncol').argmin(dim='time').values for ds in h3cn_subsets]

In [ ]:
min_slps_h3pn = [ds['PSL'].isel(time=slice(delay, clip)).compute().min('ncol').argmin(dim='time').values+5 for ds in h3pn_subsets]
min_slps_h3cn = [ds['PSL'].isel(time=slice(delay, clip)).compute().min('ncol').argmin(dim='time').values+5 for ds in h3cn_subsets]

In [ ]:
storm_names

In [ ]:
min_slps_h3pn

In [ ]:
max_fluts_h3pn

In [ ]:
storm_1521_df = return_polymesh('storm_1521', 'cam', 'FLUT', dims=dict(time=19))

In [ ]:
# Gets plot from function
cam_raster_plot = plot_native(storm_1521_df, plot_bbox=bbox, raster=True, holoviews_kw=plot_opts, 
                              coastline_kw=coastline_opts, datashader_kw=datashader_opts)
cam_raster_plot

In [ ]:
%%time
# Finds the timestep at which the maximum OLR occurs
delay = 5 # trying to avoid model spin-up
clip = -15 # trying to catch peak within middle of model run
h3pns = list(map(lambda x: all_storms[x]['h3pn_ds'], storm_names))
h3cns = list(map(lambda x: all_storms[x]['h3cn_ds'], storm_names))
max_fluts_h3pn = [ds['FLUT'].isel(time=slice(delay, clip)).compute().min('ncol').argmin(dim='time').values for ds in h3pns]
max_fluts_h3cn = [ds['FLUT'].isel(time=slice(delay, clip)).compute().min('ncol').argmin(dim='time').values for ds in h3cns]

In [ ]:
%%time
p_olr_dfs = [return_polymesh(s, 'cam', 'FLUT', dims=dict(time=i)) for s, i in zip(storm_names, max_fluts_h3pn)]
c_olr_dfs = [return_polymesh(s, 'mpas', 'FLUT', dims=dict(time=i)) for s, i in zip(storm_names, max_fluts_h3cn)]

In [ ]:
%%time
p_olr_dfs = [return_polymesh(s, 'cam', 'FLUT', dims=dict(time=i)) for s, i in zip(storm_names, min_slps_h3pn)]
c_olr_dfs = [return_polymesh(s, 'mpas', 'FLUT', dims=dict(time=i)) for s, i in zip(storm_names, min_slps_h3cn)]

## Testing storm_1279

In [ ]:
p_1279_mesh = all_storms['storm_1279']['parent_grid']
p_1279_olr = [Polymesh(p_1279_mesh, all_storms['storm_1279']['h3pn_ds'], 
                       model='cam').data_mesh(target_var='FLUT', 
                                              dims=dict(time=i), 
                                              fill="faces") for i in range(len(h3pn_subsets[-1].time))]

In [ ]:
s1279_mins = np.array([df['faces'].min() for df in p_1279_olr])

In [ ]:
s1279_mins

In [ ]:
all_storms['storm_1279']['h3pn_ds']['FLUT'].isel(time=slice(delay, clip)).compute().min('ncol').argmin(dim='time')+5

In [ ]:
s1279_mins.argmin()

In [ ]:
s1279_mins.min()

In [ ]:
# Gets plot from function
raster_plots = [plot_native(df, plot_bbox=bbox, raster=True, holoviews_kw=plot_opts, 
                              coastline_kw=coastline_opts, datashader_kw=datashader_opts) for df in p_1279_olr]

In [ ]:
times = all_storms['storm_1279']['h3cn_ds'].time.values
time_str = [ts.strftime('%m%d%Y%H') for ts in times]

In [ ]:
[hv.save(df, f'../figs/new_tracks/storm_1279/cam_olr_{t}.png', dpi=300, center=False) for df, t in zip(raster_plots, time_str)]

## Back to regular code

In [ ]:
# Defines width and height for plot
w=2000
h=1400
#h=1600

w = int(w/2)
h = int(h/2)

# Defines rasterization options (expose options with `hv.help(hds_rasterize)`)
datashader_opts = dict(aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=10)

# Defines colorbar/colormap options
flut_cmap = CustomColorbars(target_cmap='FLUT').out_cmap
flut_levs = CustomColorbars(target_cmap='FLUT').out_levels
    
cmin = flut_levs[0]
cmax = flut_levs[-1]
clabel = r'Upwelling Longwave Flux [W/m^2]'
cbar_opts = dict(height=40, border_line_width=3)
cmap_opts = dict(cmap=flut_cmap, clim=(cmin, cmax), colorbar=False, 
                 colorbar_position='bottom', clabel=clabel)

# Defines font options
# font_opts = dict(fontsize=dict(title='35pt', clabel='27pt', cticks='25pt'))
font_opts = dict(fontsize=dict(title='35pt', clabel='50pt', cticks='40pt'))
if h < 1600:
    cmap_opts['colorbar'] = False
elif h == 1600:
    cmap_opts['colorbar'] = True

# Defines plotting options (expose options with `hv.help(hv.Polygons)`)
plot_opts = dict(tools=['hover'], width=w, height=h, xaxis=None, yaxis=None,
                 colorbar_opts=cbar_opts, **cmap_opts, **font_opts)

# Defines coastline plotting options
#coastline_opts = dict(scale='10m', line_color='#f8f8f8', line_width=2.0, width=w, height=h)
lw = 2.0
coastline_opts = dict(scale='10m', line_color='#FFFFFF', line_width=lw, width=w, height=h)
states_opts = dict(scale='50m', line_color='#FFFFFF', line_width=lw, width=w, height=h, fill_color='none')

# Defines bounding bbox
bbox = basin_bboxes('florida')

# Packs into one dict for cleaner code
plotting_kwargs = {'raster':True, 'plot_bbox':bbox, 'holoviews_kw':plot_opts, 'datashader_kw':datashader_opts, 
                   'coastline_kw':coastline_opts, 'states_kw':states_opts}

In [ ]:
type(flut_cmap)

In [ ]:
type(mpl.colormaps['gist_yarg'])

In [ ]:
cam_plots = [plot_native(df, **plotting_kwargs) for df in p_olr_dfs]
mpas_plots = [plot_native(df, **plotting_kwargs) for df in c_olr_dfs]

In [ ]:
[hv.save(fig, f'../figs/new_tracks/{storm}/{storm}_peak_olr_{ts}.png', dpi=300, center=False) for fig, storm, ts in zip(cam_plots, storm_names, time_str_p)]

In [ ]:
[hv.save(fig, f'../figs/new_tracks/{storm}/{storm}_peak_olr_mpas_{ts}.png', dpi=300, center=False) for fig, storm, ts in zip(mpas_plots, storm_names, time_str_c)]

In [ ]:
# This is for making gifs

# %%time
# storm = 'storm_0236'
# times = all_storms[storm]['h3cn_ds'].time.values
# tstrings = [ts.strftime('%m%d%Y_%H') for ts in times]

# # Uses Polymesh code to create polygons from mesh data
# p_flut_dfs = [return_polymesh(storm, 'cam', 'FLUT', dims=dict(time=i)) for i in range(len(times))]

# # Uses Polymesh code to create polygons from mesh data
# c_flut_dfs = [return_polymesh(storm, 'mpas', 'FLUT', dims=dict(time=i)) for i in range(len(times))]

# title_strings = [f"{storm} {ts.strftime('%m-%d-%Y %H')}Z" for ts in times]
# cam_plots = [plot_native(df, **plotting_kwargs) for df in p_flut_dfs]
# mpas_plots = [plot_native(df, **plotting_kwargs) for df in c_flut_dfs]

# layout_kw = dict(fontsize=dict(title='45pt'))
# flut_plots = [hv.Layout([cplt, mplt]).cols(2).opts(title=t, **layout_kw) for cplt, mplt, t in zip(cam_plots, mpas_plots, title_strings)]

# tstrings = [ts.strftime('%m%d%Y%H') for ts in times]
# [hv.save(fig, f'../figs/new_tracks/{storm}/OLR_{storm}_{ts}.png', dpi=300, center=False) for fig, ts in zip(flut_plots, tstrings)]

# output_folder = r"../figs/new_tracks/storm_0236"
# import os, glob
# from PIL import Image

# # Create the frames

# imgs = glob.glob(os.path.join(output_folder, "OLR*.png"))
# imgs.sort()
# imgs = imgs[2:]

# frames = []
# for i in imgs:
#     new_frame = Image.open(i)
#     frames.append(new_frame)
    
# #Save into a GIF file that loops forever
# duration = 200
# frames[0].save(os.path.join(output_folder, f'OLR_{duration}_06052023.gif'), format='GIF',
#                append_images=frames[1:],
#                save_all=True,
#                duration=duration, loop=0)
# new_frame.close()

In [ ]:
mpas_raster_plot = plot_native(ian_df, **plotting_kwargs)

In [ ]:
# Gets plot from function
cam_raster_plot = plot_native(p_olr_dfs[-1], plot_bbox=bbox, raster=True, holoviews_kw=plot_opts, 
                              coastline_kw=coastline_opts, datashader_kw=datashader_opts)
cam_raster_plot

In [ ]:
# Gets plot from function
cam_raster_plots = [plot_native(df, plot_bbox=bbox, raster=True, holoviews_kw=plot_opts, 
                              coastline_kw=coastline_opts, datashader_kw=datashader_opts) for df in p_olr_dfs]

In [ ]:
# Gets plot from function
mpas_raster_plots = [plot_native(df, plot_bbox=bbox, raster=True, holoviews_kw=plot_opts, 
                              coastline_kw=coastline_opts, datashader_kw=datashader_opts) for df in c_olr_dfs]

In [ ]:
hv.save(mpas_raster_plot, '../figs/test_olr.png', dpi=300, center=False)

In [ ]:
[hv.save(fig, f'../figs/new_tracks/{storm}/cam_peak_olr.png', dpi=300, center=False) for fig, storm in zip(cam_raster_plots, storm_names)]

In [ ]:
[hv.save(fig, f'../figs/new_tracks/{storm}/mpas_peak_olr.png', dpi=300, center=False) for fig, storm in zip(mpas_raster_plots, storm_names)]

# Wind Swaths

In [ ]:
from uviz.datashader_tools.visualization import diverging_colormap

In [ ]:
p_sfc_dfs = list(map(return_polymesh, storm_names, ['cam']*len(storm_names), ['U10_MAX']*len(storm_names)))
c_sfc_dfs = list(map(return_polymesh, storm_names, ['mpas']*len(storm_names), ['U10_MAX']*len(storm_names)))

In [ ]:
# Defines width and height for plot
w=2000
h=1400

# Defines rasterization options (expose options with `hv.help(hds_rasterize)`)
datashader_opts = dict(aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=10)

# Gets divergent colormap for plotting
cmin=0
cmid=74
cmax=150
wsp_cmap = diverging_colormap(cmin=cmin, cmid=cmid, cmax=cmax, palette=bokeh.palettes.viridis(256))

# Defines levels for colormap ticks
cbar_levs = [0, 20, 40, 60, 80, 100, 120, 135, 150]
#cbar_levs = [0, 20, 40, 60, 80, 100, 130]
    
cmin = cbar_levs[0]
cmax = cbar_levs[-1]
clabel = 'Wind Speed [mph]'
cbar_opts = dict(height=40, scale_alpha=0.75, border_line_width=3, ticker=FixedTicker(ticks=cbar_levs))
cmap_opts = dict(cmap=wsp_cmap, clim=(cmin, cmax), colorbar=False, 
                 colorbar_position='bottom', clabel=clabel)

# Defines font options
# font_opts = dict(fontsize=dict(title='35pt', clabel='27pt', cticks='25pt'))
font_opts = dict(fontsize=dict(title='35pt', clabel='50pt', cticks='40pt'))

# Defines plotting options (expose options with `hv.help(hv.Polygons)`)
plot_opts = dict(tools=['hover'], width=w, height=h, xaxis=None, yaxis=None, alpha=0.75,
                 colorbar_opts=cbar_opts, **cmap_opts, **font_opts)

# Defines coastline plotting options
coastline_opts = dict(scale='10m', line_color='#f8f8f8', line_width=2.0, width=w, height=h)

# Defines lake plotting options
lakes_opts = dict(scale='10m', line_color='#f8f8f8', line_width=2.0, width=w, height=h, fill_color='none')

# Defines bounding bbox
bbox = basin_bboxes('florida')

In [ ]:
# Gets plot from function
cam_raster_plot = plot_native(p_sfc_dfs[0], plot_bbox=bbox, raster=True, holoviews_kw=plot_opts, 
                              coastline_kw=coastline_opts, datashader_kw=datashader_opts)
cam_raster_plot

In [ ]:
# Gets plot from function
cam_raster_plots = [plot_native(df, plot_bbox=bbox, raster=True, holoviews_kw=plot_opts, 
                              coastline_kw=coastline_opts, datashader_kw=datashader_opts, lakes_kw=lakes_opts) for df in p_sfc_dfs]

In [ ]:
# Gets plot from function
mpas_raster_plots = [plot_native(df, plot_bbox=bbox, raster=True, holoviews_kw=plot_opts, 
                              coastline_kw=coastline_opts, datashader_kw=datashader_opts, lakes_kw=lakes_opts) for df in c_sfc_dfs]

In [ ]:
[hv.save(fig, f'../figs/new_tracks/{storm}/cam_sfc_wsp.png', dpi=300, center=False) for fig, storm in zip(cam_raster_plots, storm_names)]

In [ ]:
[hv.save(fig, f'../figs/new_tracks/{storm}/mpas_sfc_wsp.png', dpi=300, center=False) for fig, storm in zip(mpas_raster_plots, storm_names)]

## 850 mb

In [ ]:
p_850_dfs = list(map(return_polymesh, storm_names, ['cam']*len(storm_names), ['WSP850_MAX']*len(storm_names)))
c_850_dfs = list(map(return_polymesh, storm_names, ['mpas']*len(storm_names), ['WSP850_MAX']*len(storm_names)))

In [ ]:
[print(df['faces'].max()) for df in p_850_dfs]

In [ ]:
[print(df['faces'].max()) for df in c_850_dfs]

In [ ]:
# Defines width and height for plot
w=2000
h=1400

# Defines rasterization options (expose options with `hv.help(hds_rasterize)`)
datashader_opts = dict(aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=10)

# Gets divergent colormap for plotting
cmin=0
cmid=100
cmax=250
wsp_cmap = diverging_colormap(cmin=cmin, cmid=cmid, cmax=cmax, palette=bokeh.palettes.viridis(256))

# Defines levels for colormap ticks
cbar_levs = [0, 50, 100, 125, 150, 175, 200, 225, 250]
#cbar_levs = [0, 20, 40, 60, 80, 100, 130]
    
cmin = cbar_levs[0]
cmax = cbar_levs[-1]
clabel = 'Wind Speed [mph]'
cbar_opts = dict(height=40, scale_alpha=0.75, border_line_width=3, ticker=FixedTicker(ticks=cbar_levs))
cmap_opts = dict(cmap=wsp_cmap, clim=(cmin, cmax), colorbar=False, 
                 colorbar_position='bottom', clabel=clabel)

# Defines font options
# font_opts = dict(fontsize=dict(title='35pt', clabel='27pt', cticks='25pt'))
font_opts = dict(fontsize=dict(title='35pt', clabel='50pt', cticks='40pt'))

# Defines plotting options (expose options with `hv.help(hv.Polygons)`)
plot_opts = dict(tools=['hover'], width=w, height=h, xaxis=None, yaxis=None, alpha=0.75,
                 colorbar_opts=cbar_opts, **cmap_opts, **font_opts)

# Defines coastline plotting options
coastline_opts = dict(scale='10m', line_color='#f8f8f8', line_width=2.0, width=w, height=h)

# Defines lake plotting options
lakes_opts = dict(scale='10m', line_color='#f8f8f8', line_width=2.0, width=w, height=h, fill_color='none')

# Defines bounding bbox
bbox = basin_bboxes('florida')

In [ ]:
# Gets plot from function
cam_raster_plot = plot_native(p_850_dfs[0], plot_bbox=bbox, raster=True, holoviews_kw=plot_opts, 
                              coastline_kw=coastline_opts, datashader_kw=datashader_opts)
cam_raster_plot

In [ ]:
# Gets plot from function
cam_raster_plots = [plot_native(df, plot_bbox=bbox, raster=True, holoviews_kw=plot_opts, 
                              coastline_kw=coastline_opts, datashader_kw=datashader_opts, lakes_kw=lakes_opts) for df in p_850_dfs]

In [ ]:
# Gets plot from function
mpas_raster_plots = [plot_native(df, plot_bbox=bbox, raster=True, holoviews_kw=plot_opts, 
                              coastline_kw=coastline_opts, datashader_kw=datashader_opts, lakes_kw=lakes_opts) for df in c_850_dfs]

In [ ]:
[hv.save(fig, f'../figs/new_tracks/{storm}/cam_850_wsp.png', dpi=300, center=False) for fig, storm in zip(cam_raster_plots, storm_names)]

In [ ]:
[hv.save(fig, f'../figs/new_tracks/{storm}/mpas_850_wsp.png', dpi=300, center=False) for fig, storm in zip(mpas_raster_plots, storm_names)]

# Precip Swaths

## Masking Land

In [ ]:
from sklearn.neighbors import KDTree, BallTree

def kd_tree(locations1, locations2, metric='euclidean', neighbors=1):
    kd = KDTree(locations1, metric=metric)
    dists, idx = kd.query(locations2, k=neighbors)
    
    return idx

def ball_tree(locations1, locations2, metric='haversine', neighbors=1, leaf_size=15):
    
    # Converts degrees to radians
    locations1 = np.deg2rad(locations1)
    locations2 = np.deg2rad(locations2)
    
    ball = BallTree(locations1, metric=metric, leaf_size=leaf_size)
    dists, idx = ball.query(locations2, k=neighbors)
    
    return idx

def get_landmask(ds, topo_ds):
    
    # Finds landmask indices
    land_lat_idx, land_lon_idx = np.where(topo_ds['LANDFRAC'].values > 0.5)

    # Finds landmask coordinates
    land_lats = topo_ds.lat.values[land_lat_idx]
    land_lons = topo_ds.lon.values[land_lon_idx]
    land_lons = np.mod(land_lons, 180) - 180
    
    # Data coordinates
    lats = ds['lat'].isel(time=0).values
    lons = ds['lon'].isel(time=0).values
    lons = np.mod(lons, 180) - 180
    
    # Reshapes coordinates into proper format
    ds_coords = np.dstack([lats, lons]).reshape(-1, 2)
    land_coords = np.dstack([land_lats, land_lons]).reshape(-1, 2)
    
    # Gets nearest neighbor indices
    nn_idx = kd_tree(ds_coords, land_coords)
    #nn_idx = ball_tree(ds_coords, land_coords)
    
    # Gets unique values and flattens array
    idx = np.unique(nn_idx.flatten())
    
    return idx

In [ ]:
def mask_land(ds, return_idx=False):
    
    # Checks if MPAS or CAM based on number of grid cells
    if len(ds.ncol) == 835586:
        model = 'MPAS'
    else:
        model = 'CAM'
        
    # Masks out land grid cells
    if model == 'MPAS':
        idx = mpas_idx
    elif model == 'CAM':
        if len(ds.ncol) == 155981:
            idx = ext_idx
        elif len(ds.ncol) == 119603:
            idx = ref_idx
        elif len(ds.ncol) == 69653:
            idx = wat_idx
            
    # Selects by land indices
    ds = ds.isel(ncol=idx)
    
    # Returns indices if requested
    if return_idx == True:
        return ds, idx
    else:
        return ds

In [ ]:
# Reads in landmask datasets
landmask_dir = r'/gpfs/group/cmz5202/default/cnd5285/landmasks'

mpas_land_ds = xr.open_dataset(os.path.join(landmask_dir, 'MPAS.VR3_landmask.nc'))
ext_land_ds = xr.open_dataset(os.path.join(landmask_dir, 'SE.VR28.NATL.EXT_landmask.nc'))
ref_land_ds = xr.open_dataset(os.path.join(landmask_dir, 'SE.VR28.NATL.REF_landmask.nc'))
wat_land_ds = xr.open_dataset(os.path.join(landmask_dir, 'SE.VR28.NATL.WAT_landmask.nc'))

In [ ]:
# Gets indices of overland masks
mpas_idx = np.where(mpas_land_ds['LANDFRAC'] >= 0.5)[0]
ext_idx = np.where(ext_land_ds['LANDFRAC'] >= 0.5)[0]
ref_idx = np.where(ref_land_ds['LANDFRAC'] >= 0.5)[0]
wat_idx = np.where(wat_land_ds['LANDFRAC'] >= 0.5)[0]

In [ ]:
# %%time
# # Block to gather overland CAM grids
# print('Reading in topography dataset.')
# topo_ds = xr.open_dataset('/storage/work/cmz5202/cam_tools/hires-topo/2min_cesm_topo_latlon.nc')

# %%time
# # Gets cubed-sphere indices of overland mask
# print('Getting indices of CAM land cells')
# ext_idx = get_landmask(all_storms['Irma']['h4pn_ds'], topo_ds)
# ref_idx = get_landmask(all_storms['storm_1279']['h4pn_ds'], topo_ds)
# wat_idx = get_landmask(all_storms['storm_0310']['h4pn_ds'], topo_ds)

# # Gets hexagonal indices of overland mask
# print('Getting indices of MPAS land cells')
# mpas_idx = get_landmask(all_storms['Irma']['h4cn_ds'], topo_ds)

In [ ]:
# Colin defined bounding boxes (old)
# irma_charley_bbox = basin_bboxes({'west_coord':-82.335520, 'east_coord':-80.042506, 'north_coord':26.919435, 'south_coord':25.137884})
# ian_bbox = basin_bboxes({'west_coord':-81.966989, 'east_coord':-80.904318, 'north_coord':28.319877, 'south_coord':27.662304})
# ian_bbox = basin_bboxes({'west_coord':-81.966989, 'east_coord':-80.904318, 'north_coord':28.319877, 'south_coord':27.662304})
# isaac_fay_bbox = (basin_bboxes({'west_coord':-82.335520, 'east_coord':-80.042506,'north_coord':26.919435, 'south_coord':25.137884}))

# Colin defined bounding boxes (old2)
# irma_charley_bbox = basin_bboxes({'west_coord':-82.335520, 'east_coord':-80.042506, 'north_coord':26.919435, 'south_coord':25.137884})
# isaac_bbox = (basin_bboxes({'west_coord':-82.335520, 'east_coord':-80.042506,'north_coord':26.919435, 'south_coord':25.137884}))
# fay_bbox = (basin_bboxes({'west_coord':-81.717809, 'east_coord':-80.096434,'north_coord':28.686758, 'south_coord':27.039041}))
# ian_bbox = basin_bboxes({'west_coord':-82.857622, 'east_coord':-81.090739, 'north_coord':27.921146, 'south_coord':26.627882})

In [ ]:
# Corrine defined bounding boxes
irma_bbox = basin_bboxes({'west_coord':-82.335520, 'east_coord':-80.042506, 'south_coord':25.137884, 'north_coord':26.919435})
charley_bbox = basin_bboxes({'west_coord':-82.857622, 'east_coord':-80.096434, 'south_coord':26.124555, 'north_coord':28.716921})
isaac_bbox = basin_bboxes({'west_coord':-82.335520, 'east_coord':-80.042506, 'south_coord':25.137884, 'north_coord':26.919435})
fay_bbox = irma_bbox
#fay_bbox = basin_bboxes({'west_coord':-81.717809, 'east_coord':-80.096434, 'south_coord':26.610566, 'north_coord':29.191563})
ian_bbox = basin_bboxes({'west_coord':-82.857622, 'east_coord':-81.090739, 'south_coord':26.426490, 'north_coord':28.165928})

In [ ]:
ian_pidx = get_subset_idx(all_storms['Ian']['h4pn_ds'], ian_bbox)
ian_cidx = get_subset_idx(all_storms['Ian']['h4cn_ds'], ian_bbox)

irma_pidx = get_subset_idx(all_storms['Irma']['h4pn_ds'], irma_bbox)
irma_cidx = get_subset_idx(all_storms['Irma']['h4cn_ds'], irma_bbox)

# charley_pidx = get_subset_idx(all_storms['Charley']['h4pn_ds'], irma_charley_bbox)
# charley_cidx = get_subset_idx(all_storms['Charley']['h4cn_ds'], irma_charley_bbox)

fay_pidx = get_subset_idx(all_storms['Fay']['h4pn_ds'], fay_bbox)
fay_cidx = get_subset_idx(all_storms['Fay']['h4cn_ds'], fay_bbox)

isaac_pidx = get_subset_idx(all_storms['Isaac']['h4pn_ds'], isaac_bbox)
isaac_cidx = get_subset_idx(all_storms['Isaac']['h4cn_ds'], isaac_bbox)

In [ ]:
hist_storms = [x for x in all_storms.keys() if x in ['Irma', 'Ian', 'Fay', 'Isaac']]
sim_storms = [x for x in all_storms.keys() if x not in ['Irma', 'Ian', 'Charley', 'Fay', 'Isaac']]

In [ ]:
storm_names

In [ ]:
# Uses Polymesh code to create polygons from mesh data
p_rprect_dfs = list(map(return_polymesh, storm_names, ['cam']*len(storm_names), ['PRECT_MAX']*len(storm_names)))
c_rprect_dfs = list(map(return_polymesh, storm_names, ['mpas']*len(storm_names), ['PRECT_MAX']*len(storm_names)))

In [ ]:
# p_idx = [ian_pidx, irma_pidx, charley_pidx, fay_pidx, isaac_pidx]
# c_idx = [ian_cidx, irma_cidx, charley_cidx, fay_cidx, isaac_cidx]

p_idx = [ian_pidx, irma_pidx, fay_pidx, isaac_pidx]
c_idx = [ian_cidx, irma_cidx, fay_cidx, isaac_cidx]

p_tprect_dfs = list(map(return_polymesh, hist_storms, ['cam']*len(storm_names), 
                        ['PRECT_TOT']*len(hist_storms), p_idx))
c_tprect_dfs = list(map(return_polymesh, hist_storms, ['mpas']*len(storm_names), 
                        ['PRECT_TOT']*len(hist_storms), c_idx))

In [ ]:
# Uses Polymesh code to create polygons from mesh data
p_tprect_dfs = list(map(return_polymesh, hist_storms, ['cam']*len(hist_storms), ['PRECT_TOT']*len(hist_storms)))
p_rprect_dfs = list(map(return_polymesh, hist_storms, ['cam']*len(hist_storms), ['PRECT_MAX']*len(hist_storms)))

c_tprect_dfs = list(map(return_polymesh, hist_storms, ['mpas']*len(hist_storms), ['PRECT_TOT']*len(hist_storms)))
c_rprect_dfs = list(map(return_polymesh, hist_storms, ['mpas']*len(hist_storms), ['PRECT_MAX']*len(hist_storms)))

In [ ]:
# Defines width and height for plot (sans colorbar)
# w=2000
# h=1600

# This size is for quick plotting in jupyter
#w = 1000
#h = 700

# For exporting with colorbar
# w=2000
# h=1400

# This size is for just the really zoomed in Florida plot
w = 800
h = 700

# Defines variable for plotting
var = 'PRECT_TOT'

# Defines rasterization options (expose options with `hv.help(hds_rasterize)`)
datashader_opts = dict(aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=10)

# Defines colorbar/colormap options
prect_cmap = CustomColorbars('nws_precip').out_cmap
prect_levels = CustomColorbars('nws_precip').out_levels
prect_norm = CustomColorbars('nws_precip').norm

if var == 'PRECT_TOT':
    cbar_levs = [0, 2, 4, 6, 8, 10, 15, 20, 25, 30, 35, 40, 45, 50]
    cbar_opts = dict(height=40, scale_alpha=0.75, border_line_width=3, ticker=FixedTicker(ticks=cbar_levs))
    clabel = 'Total Precipitation [inches]'
elif var == 'PRECT_MAX':
    #cbar_levs = [0, 0.5, 1, 2, 3, 4, 5, 6, 8]
    cbar_levs = [0, 0.25, 0.5, 1, 2, 3, 4, 5]
    clabel = 'Maximum Hourly Rate [inches]'
    #prect_levels = prect_levels[:11]
    prect_levels = prect_levels[:9]
    # prect_cmap = ["#ffffff", "#ffffff", "#4bd2f7", "#699fd0", "#3c4bac", "#3cf74b", 
    #               "#3cb447", "#3c8743", "#1f4723", "#f7f73c"]
    prect_cmap = ['#ffffff', '#ffffff', "#4bd2f7", "#699fd0", "#3c4bac", "#3cf74b", "#3cb447", "#3c8743"]
    cbar_opts = dict(height=40, scale_alpha=0.75, border_line_width=3, ticker=FixedTicker(ticks=cbar_levs))
    
cmin = prect_levels[0]
cmax = prect_levels[-1]

#cbar_opts = dict(height=40, scale_alpha=0.75, border_line_width=3)
if h < 1600:
    cmap_opts = dict(cmap=prect_cmap, clim=(cmin, cmax), colorbar=False)
elif h == 1600:
    cmap_opts = dict(cmap=prect_cmap, clim=(cmin, cmax), colorbar=True, 
                     colorbar_position='bottom', clabel=clabel)

# Defines font options
# font_opts = dict(fontsize=dict(title='35pt', clabel='27pt', cticks='25pt'))
font_opts = dict(fontsize=dict(title='35pt', clabel='50pt', cticks='40pt'))

# Defines plotting options (expose options with `hv.help(hv.Polygons)`)
plot_opts = dict(tools=['hover'], width=w, height=h, xaxis=None, yaxis=None, alpha=0.75, color_levels=prect_levels,
                 colorbar_opts=cbar_opts, **cmap_opts, **font_opts)

# Defines coastline plotting options
#coastline_opts = dict(scale='10m', line_color='#323232', line_width=2.0, width=w, height=h)
coastline_opts = dict(scale='10m', line_color='#000000', line_width=2.0, width=w, height=h)

# Defines lake plotting options
#lakes_opts = dict(scale='10m', line_color='#616161', line_width=2.0, width=w, height=h, fill_color='#e4f1fa')
lakes_opts = dict(scale='10m', line_color='#616161', line_width=2.0, width=w, height=h, fill_color='none')

# Defines ocean plotting options
ocean_opts = dict(scale='10m', line_color='#e4f1fa', line_width=2.0, width=w, height=h, fill_color='#e4f1fa')

# Defines state plotting options
#states_opts = dict(scale='10m', line_color='#616161', line_width=1.5, width=w, height=h, fill_color='#EBEBEB')
states_opts = dict(scale='10m', line_color='#616161', line_width=1.5, width=w, height=h, fill_color='none')

# Defines plotting bounding bbox
#bbox = basin_bboxes('florida')

bbox = basin_bboxes({'west_coord':-88.5, 'east_coord':-79.0, 'south_coord':24, 'north_coord':32.5})

In [ ]:
def project_bbox(bbox, proj=ccrs.PlateCarree()):
    lon_range, lat_range = bbox
    x_range, y_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
    new_bbox = tuple((x_range[0], y_range[0], x_range[1], y_range[1]))
    
    return new_bbox

rect_bboxes = list(map(project_bbox, [ian_bbox, irma_bbox, fay_bbox, isaac_bbox]))

In [ ]:
# Gets plot from function
native_opts = {'holoviews_kw':plot_opts, 'datashader_kw':datashader_opts, 
               'coastline_kw':coastline_opts, 'lakes_kw':lakes_opts, 
               'ocean_kw':ocean_opts, 'states_kw':states_opts}
native_opts = {'holoviews_kw':plot_opts, 'datashader_kw':datashader_opts, 
               'coastline_kw':coastline_opts, 'lakes_kw':lakes_opts, 'states_kw':states_opts}
mpas_raster_plot = plot_native(c_tprect_dfs[1], plot_bbox=bbox, raster=True, **native_opts)
mpas_raster_plot * hv.Rectangles(project_bbox(irma_bbox)).opts(fill_color='none', color='#000000', line_width=2.5)

In [ ]:
# Gets plot from function
cam_raster_plot = plot_native(p_tprect_dfs[0], plot_bbox=bbox, raster=True, holoviews_kw=plot_opts, 
                              coastline_kw=coastline_opts, datashader_kw=datashader_opts, lakes_kw=lakes_opts, 
                              ocean_kw=ocean_opts, states_kw=states_opts)
cam_raster_plot

In [ ]:
# Gets plot from function
cam_raster_plots = [plot_native(df, plot_bbox=bbox, raster=True, holoviews_kw=plot_opts, 
                                coastline_kw=coastline_opts, datashader_kw=datashader_opts, 
                                lakes_kw=lakes_opts, ocean_kw=ocean_opts, states_kw=states_opts) for df in p_tprect_dfs]

In [ ]:
# Gets plot from function
mpas_raster_plots = [plot_native(df, plot_bbox=bbox, raster=True, holoviews_kw=plot_opts, 
                                 coastline_kw=coastline_opts, datashader_kw=datashader_opts, 
                                 lakes_kw=lakes_opts, ocean_kw=ocean_opts, states_kw=states_opts) for df in c_tprect_dfs]

In [ ]:
[hv.save(fig, f'../figs/new_tracks/{storm}/mpas_prect_overland.png', dpi=300, center=False) for fig, storm in zip(mpas_raster_plots, storm_names)]

In [ ]:
[hv.save(fig, f'../figs/new_tracks/{storm}/cam_prect_overland.png', dpi=300, center=False) for fig, storm in zip(cam_raster_plots, storm_names)]

## Regular Code

In [ ]:
#storm_names = [x for x in all_storms.keys() if x not in ['Ian', 'Irma', 'Isaac', 'Fay', 'Charley']]

# Uses Polymesh code to create polygons from mesh data
p_tprect_dfs = list(map(return_polymesh, storm_names, ['cam']*len(storm_names), ['PRECT_TOT']*len(storm_names)))
p_rprect_dfs = list(map(return_polymesh, storm_names, ['cam']*len(storm_names), ['PRECT_MAX']*len(storm_names)))

c_tprect_dfs = list(map(return_polymesh, storm_names, ['mpas']*len(storm_names), ['PRECT_TOT']*len(storm_names)))
c_rprect_dfs = list(map(return_polymesh, storm_names, ['mpas']*len(storm_names), ['PRECT_MAX']*len(storm_names)))

In [ ]:
p_rprect_df = return_polymesh('storm_0236', 'cam', 'PRECT_MAX')

In [ ]:
def get_precip_kwargs(var, colorbar=False):
    
    w=2000
    h=1400
    
    w = int(w/2)
    h = int(h/2)

    # Defines variable for plotting
    #var = 'PRECT_MAX'

    # Defines rasterization options (expose options with `hv.help(hds_rasterize)`)
    datashader_opts = dict(aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=10)

    # Defines colorbar/colormap options
    prect_cmap = CustomColorbars('nws_precip').out_cmap
    prect_levels = CustomColorbars('nws_precip').out_levels
    prect_norm = CustomColorbars('nws_precip').norm

    if var == 'PRECT_TOT':
        cbar_levs = [0, 2, 4, 6, 8, 10, 15, 20, 25, 30, 35, 40, 45, 50]
        cbar_opts = dict(height=40, scale_alpha=0.75, border_line_width=3, ticker=FixedTicker(ticks=cbar_levs))
        clabel = 'Total Precipitation [inches]'
    elif var == 'PRECT_MAX':
        cbar_levs = [0, 0.25, 0.5, 1, 2, 3, 4, 5, 6, 8]
        #cbar_levs = [0, 0.25, 0.5, 1, 2, 3, 4, 5]
        clabel = 'Accumulated Precipitation [inches]'
        #prect_levels = prect_levels[:11]
        prect_levels = prect_levels[:11]
        # prect_cmap = ["#ffffff", "#ffffff", "#4bd2f7", "#699fd0", "#3c4bac", "#3cf74b", 
        #               "#3cb447", "#3c8743", "#1f4723", "#f7f73c"]
        #prect_cmap = ['#ffffff', '#ffffff', "#4bd2f7", "#699fd0", "#3c4bac", "#3cf74b", "#3cb447", "#3c8743"]
        prect_cmap = ['#ffffff', '#ffffff', "#4bd2f7", "#699fd0", "#3c4bac", "#3cf74b", "#3cb447", "#3c8743", "#1f4723", "#f7f73c"]
        cbar_opts = dict(height=40, scale_alpha=0.75, border_line_width=3, ticker=FixedTicker(ticks=cbar_levs))

    cmin = prect_levels[0]
    cmax = prect_levels[-1]

    #cbar_opts = dict(height=40, scale_alpha=0.75, border_line_width=3)
    if colorbar == False:
        cmap_opts = dict(cmap=prect_cmap, clim=(cmin, cmax), colorbar=False)
    elif colorbar == True:
        w = 2000
        h = 1600
        cmap_opts = dict(cmap=prect_cmap, clim=(cmin, cmax), colorbar=True, 
                         colorbar_position='bottom', clabel=clabel)
        lakes_opts = {}
        states_opts = {}

    # Defines font options
    # font_opts = dict(fontsize=dict(title='35pt', clabel='27pt', cticks='25pt'))
    font_opts = dict(fontsize=dict(title='35pt', clabel='50pt', cticks='40pt'))

    # Defines plotting options (expose options with `hv.help(hv.Polygons)`)
    plot_opts = dict(tools=['hover'], width=w, height=h, xaxis=None, yaxis=None, alpha=0.75, color_levels=prect_levels,
                     colorbar_opts=cbar_opts, **cmap_opts, **font_opts)

    # Defines coastline plotting options
    #coastline_opts = dict(scale='10m', line_color='#323232', line_width=3.0, width=w, height=h)
    lw=3.0
    coastline_opts = dict(scale='10m', line_color='#000000', line_width=lw, width=w, height=h)

    # Defines lake plotting options
    #lakes_opts = dict(scale='10m', line_color='#323232', line_width=4.0, width=w, height=h, fill_color='none')
    #states_opts = dict(scale='50m', line_color='#616161', line_width=lw, width=w, height=h, fill_color='none')
    
    # These
    # lakes_opts = dict(scale='10m', line_color='#000000', line_width=lw, width=w, height=h, fill_color='none')
    # states_opts = dict(scale='50m', line_color='#000000', line_width=lw, width=w, height=h, fill_color='none')
    # ocean_opts = dict(scale='10m', line_color='#e4f1fa', width=w, height=h, fill_color='#e4f1fa')
    

    # Defines bounding bbox
    bbox = basin_bboxes('florida')

    # Packs into one dict for cleaner code
    plotting_kwargs = {'plot_bbox':bbox, 'raster':True, 'holoviews_kw':plot_opts, 'datashader_kw':datashader_opts, 
                       'coastline_kw':coastline_opts, 'lakes_kw':lakes_opts, 'states_kw':states_opts}
    
    return plotting_kwargs

In [ ]:
# Defines width and height for plot
w=2000
h=1400

w = int(w/2)
h = int(h/2)

# Defines variable for plotting
var = 'PRECT_MAX'

# Defines rasterization options (expose options with `hv.help(hds_rasterize)`)
datashader_opts = dict(aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=10)

# Defines colorbar/colormap options
prect_cmap = CustomColorbars('nws_precip').out_cmap
prect_levels = CustomColorbars('nws_precip').out_levels
prect_norm = CustomColorbars('nws_precip').norm

if var == 'PRECT_TOT':
    cbar_levs = [0, 2, 4, 6, 8, 10, 15, 20, 25, 30, 35, 40, 45, 50]
    cbar_opts = dict(height=40, scale_alpha=0.75, border_line_width=3, ticker=FixedTicker(ticks=cbar_levs))
    clabel = 'Total Precipitation [inches]'
elif var == 'PRECT_MAX':
    #cbar_levs = [0, 0.5, 1, 2, 3, 4, 5, 6, 8]
    cbar_levs = [0, 0.25, 0.5, 1, 2, 3, 4, 5]
    clabel = 'Accumulated Precipitation [inches]'
    #prect_levels = prect_levels[:11]
    prect_levels = prect_levels[:9]
    # prect_cmap = ["#ffffff", "#ffffff", "#4bd2f7", "#699fd0", "#3c4bac", "#3cf74b", 
    #               "#3cb447", "#3c8743", "#1f4723", "#f7f73c"]
    prect_cmap = ['#ffffff', '#ffffff', "#4bd2f7", "#699fd0", "#3c4bac", "#3cf74b", "#3cb447", "#3c8743"]
    cbar_opts = dict(height=40, scale_alpha=0.75, border_line_width=3, ticker=FixedTicker(ticks=cbar_levs))
    
cmin = prect_levels[0]
cmax = prect_levels[-1]

#cbar_opts = dict(height=40, scale_alpha=0.75, border_line_width=3)
if h < 1600:
    cmap_opts = dict(cmap=prect_cmap, clim=(cmin, cmax), colorbar=False)
elif h == 1600:
    cmap_opts = dict(cmap=prect_cmap, clim=(cmin, cmax), colorbar=True, 
                     colorbar_position='bottom', clabel=clabel)

# Defines font options
# font_opts = dict(fontsize=dict(title='35pt', clabel='27pt', cticks='25pt'))
font_opts = dict(fontsize=dict(title='35pt', clabel='50pt', cticks='40pt'))

# Defines plotting options (expose options with `hv.help(hv.Polygons)`)
plot_opts = dict(tools=['hover'], width=w, height=h, xaxis=None, yaxis=None, alpha=0.75, color_levels=prect_levels,
                 colorbar_opts=cbar_opts, **cmap_opts, **font_opts)

# Defines coastline plotting options
#coastline_opts = dict(scale='10m', line_color='#323232', line_width=3.0, width=w, height=h)
lw=3.0
coastline_opts = dict(scale='10m', line_color='#000000', line_width=lw, width=w, height=h)

# Defines lake plotting options
#lakes_opts = dict(scale='10m', line_color='#323232', line_width=4.0, width=w, height=h, fill_color='none')
lakes_opts = dict(scale='10m', line_color='#000000', line_width=lw, width=w, height=h, fill_color='none')
states_opts = dict(scale='50m', line_color='#000000', line_width=lw, width=w, height=h, fill_color='none')
#states_opts = dict(scale='50m', line_color='#616161', line_width=lw, width=w, height=h, fill_color='none')
ocean_opts = dict(scale='10m', line_color='#e4f1fa', width=w, height=h, fill_color='#e4f1fa')

# Defines bounding bbox
bbox = basin_bboxes('florida')

# Packs into one dict for cleaner code
plotting_kwargs = {'plot_bbox':bbox, 'raster':True, 'holoviews_kw':plot_opts, 'datashader_kw':datashader_opts, 
                   'coastline_kw':coastline_opts, 'lakes_kw':lakes_opts, 'states_kw':states_opts}

# Defines custom bbox to match nexrad figs
#bbox = basin_bboxes({'west_coord':-88.5, 'east_coord':-79, 'south_coord':24, 'north_coord':

In [ ]:
hist_storms = [x for x in all_storms.keys() if x in ['Irma', 'Ian', 'Fay', 'Isaac']]
sim_storms = [x for x in all_storms.keys() if x not in ['Irma', 'Ian', 'Charley', 'Fay', 'Isaac']]

hist_storms

In [ ]:
irma_bbox = basin_bboxes({'west_coord':-82.335520, 'east_coord':-80.042506, 'south_coord':25.137884, 'north_coord':26.919435})
charley_bbox = basin_bboxes({'west_coord':-82.857622, 'east_coord':-80.096434, 'south_coord':26.124555, 'north_coord':28.716921})
isaac_bbox = basin_bboxes({'west_coord':-82.335520, 'east_coord':-80.042506, 'south_coord':25.137884, 'north_coord':26.919435})
fay_bbox = irma_bbox
ian_bbox = basin_bboxes({'west_coord':-82.857622, 'east_coord':-81.090739, 'south_coord':26.426490, 'north_coord':28.165928})

storm_0236_bbox = basin_bboxes({'west_coord':-82.733860, 'east_coord':-80.036716, 'south_coord':25.121120, 'north_coord':27.495256})
storm_0755_bbox = basin_bboxes({'west_coord':-82.733860, 'east_coord':-80.036716, 'south_coord':25.348799, 'north_coord':27.777070})
storm_0528_bbox = basin_bboxes({'west_coord':-82.733860, 'east_coord':-80.036716, 'south_coord':25.121120, 'north_coord':27.495256})
storm_1048_bbox = basin_bboxes({'west_coord':-82.858682, 'east_coord':-80.036716, 'south_coord':25.629735, 'north_coord':28.508998})
storm_1521_bbox = basin_bboxes({'west_coord':-82.733860, 'east_coord':-80.036716, 'south_coord':25.121120, 'north_coord':27.946685})
storm_1354_bbox = basin_bboxes({'west_coord':-81.992647, 'east_coord':-80.036716, 'south_coord':25.121120, 'north_coord':26.700814})
storm_1307_bbox = basin_bboxes({'west_coord':-82.515101, 'east_coord':-80.036716, 'south_coord':25.121120, 'north_coord':27.210623})
storm_0310_bbox = basin_bboxes({'west_coord':-82.861571, 'east_coord':-80.036716, 'south_coord':25.659481, 'north_coord':28.412060})
storm_1279_bbox = basin_bboxes({'west_coord':-82.733860, 'east_coord':-80.036716, 'south_coord':25.121120, 'north_coord':27.357253})

In [ ]:
if len(storm_names) == 13:
    bboxes = [ian_bbox, irma_bbox, storm_0236_bbox, storm_0755_bbox, 
              storm_0528_bbox, storm_1048_bbox, storm_1521_bbox, 
              fay_bbox, isaac_bbox, storm_1354_bbox, storm_1307_bbox, 
              storm_0310_bbox, storm_1279_bbox]
elif len(storm_names) == 9:
    bboxes = [storm_0236_bbox, storm_0755_bbox, storm_0528_bbox, 
              storm_1048_bbox, storm_1521_bbox, storm_1354_bbox, 
              storm_1307_bbox, storm_0310_bbox, storm_1279_bbox]
elif len(storm_names) == 4:
    bboxes = [ian_bbox, irma_bbox, fay_bbox, isaac_bbox]

In [ ]:
def project_bbox(bbox, proj=ccrs.PlateCarree()):
    lon_range, lat_range = bbox
    x_range, y_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
    new_bbox = tuple((x_range[0], y_range[0], x_range[1], y_range[1]))
    
    return new_bbox

rect_bboxes = list(map(project_bbox, bboxes))

In [ ]:
# Gets plot from function
mpas_raster_plot = plot_native(c_tprect_dfs[1], plot_bbox=bbox, raster=True, holoviews_kw=plot_opts, 
                              coastline_kw=coastline_opts, datashader_kw=datashader_opts, lakes_kw=lakes_opts)
mpas_raster_plot * hv.Rectangles(project_bbox(isaac_bbox)).opts(fill_color='none', color='#000000', line_width=4)

In [ ]:
# Gets plot from function
cam_raster_plot = plot_native(p_rprect_dfs[0], plot_bbox=bbox, raster=True, holoviews_kw=plot_opts, 
                              coastline_kw=coastline_opts, datashader_kw=datashader_opts, lakes_kw=lakes_opts)
cam_raster_plot

In [ ]:
# Gets plot from function
plotting_kwargs = get_precip_kwargs('PRECT_MAX', True)
cam_raster_plot = plot_native(p_rprect_df, **plotting_kwargs)
hv.save(cam_raster_plot, f'../figs/new_tracks/storm_0236/cam_rate_colorbar.png', dpi=300, center=False)

### Rate with Bboxes

In [ ]:
# Gets plot from function
plotting_kwargs = get_precip_kwargs('PRECT_MAX')
cam_raster_plots = [plot_native(df, **plotting_kwargs) for df in p_rprect_dfs]
cam_raster_plots = [plot * hv.Rectangles(bbox).opts(fill_color='none', color='#000000', line_width=10) for plot, bbox in zip(cam_raster_plots, rect_bboxes)]
[hv.save(fig, f'../figs/new_tracks/{storm}/cam_rate_prect_bboxes.png', dpi=300, center=False) for fig, storm in zip(cam_raster_plots, storm_names)]

In [ ]:
# Gets plot from function
plotting_kwargs = get_precip_kwargs('PRECT_MAX')
mpas_raster_plots = [plot_native(df, **plotting_kwargs) for df in c_rprect_dfs]
mpas_raster_plots = [plot * hv.Rectangles(bbox).opts(fill_color='none', color='#000000', line_width=10) for plot, bbox in zip(mpas_raster_plots, rect_bboxes)]
[hv.save(fig, f'../figs/new_tracks/{storm}/mpas_rate_prect_bboxes.png', dpi=300, center=False) for fig, storm in zip(mpas_raster_plots, storm_names)]

### Total with Bboxes

In [ ]:
# Gets plot from function
plotting_kwargs = get_precip_kwargs('PRECT_TOT')
cam_raster_plots = [plot_native(df, **plotting_kwargs) for df in p_tprect_dfs]
cam_raster_plots = [plot * hv.Rectangles(bbox).opts(fill_color='none', color='#000000', line_width=10) for plot, bbox in zip(cam_raster_plots, rect_bboxes)]
[hv.save(fig, f'../figs/new_tracks/{storm}/cam_total_prect_bboxes.png', dpi=300, center=False) for fig, storm in zip(cam_raster_plots, storm_names)]

In [ ]:
# Gets plot from function
plotting_kwargs = get_precip_kwargs('PRECT_TOT')
mpas_raster_plots = [plot_native(df, **plotting_kwargs) for df in c_tprect_dfs]
mpas_raster_plots = [plot * hv.Rectangles(bbox).opts(fill_color='none', color='#000000', line_width=10) for plot, bbox in zip(mpas_raster_plots, rect_bboxes)]
[hv.save(fig, f'../figs/new_tracks/{storm}/mpas_total_prect_bboxes.png', dpi=300, center=False) for fig, storm in zip(mpas_raster_plots, storm_names)]

### Rate without Bboxes

In [ ]:
# Gets plot from function
plotting_kwargs = get_precip_kwargs('PRECT_MAX')
cam_raster_plots = [plot_native(df, **plotting_kwargs) for df in p_rprect_dfs]
[hv.save(fig, f'../figs/new_tracks/{storm}/cam_rate_prect.png', dpi=300, center=False) for fig, storm in zip(cam_raster_plots, storm_names)]

In [ ]:
# Gets plot from function
plotting_kwargs = get_precip_kwargs('PRECT_MAX')
mpas_raster_plots = [plot_native(df, **plotting_kwargs) for df in c_rprect_dfs]
[hv.save(fig, f'../figs/new_tracks/{storm}/mpas_rate_prect.png', dpi=300, center=False) for fig, storm in zip(mpas_raster_plots, storm_names)]

### Total without Bboxes

In [ ]:
# Gets plot from function
plotting_kwargs = get_precip_kwargs('PRECT_TOT')
cam_raster_plots = [plot_native(df, **plotting_kwargs) for df in p_tprect_dfs]
[hv.save(fig, f'../figs/new_tracks/{storm}/cam_total_prect.png', dpi=300, center=False) for fig, storm in zip(cam_raster_plots, storm_names)]

In [ ]:
# Gets plot from function
plotting_kwargs = get_precip_kwargs('PRECT_TOT')
mpas_raster_plots = [plot_native(df, **plotting_kwargs) for df in c_tprect_dfs]
[hv.save(fig, f'../figs/new_tracks/{storm}/mpas_total_prect.png', dpi=300, center=False) for fig, storm in zip(mpas_raster_plots, storm_names)]

# CAPE

In [ ]:
%%time
h4pn_subsets = list(map(lambda x: spatial_subset(all_storms[x]['h4pn_ds'], 'florida'), storm_names))
h4cn_subsets = list(map(lambda x: spatial_subset(all_storms[x]['h4cn_ds'], 'florida'), storm_names))

In [ ]:
delay=5
clip=-1
max_precip_h4pn = [ds['PRECT'].isel(time=slice(delay, clip)).compute().max('ncol').argmax(dim='time').values+delay for ds in h4pn_subsets]
max_precip_h4cn = [ds['PRECT'].isel(time=slice(delay, clip)).compute().max('ncol').argmax(dim='time').values+delay for ds in h4cn_subsets]

In [ ]:
max_precip_h4pn

In [ ]:
[int(x/3) for x in max_precip_h4pn]

In [ ]:
max_precip_h4cn

In [ ]:
[int(x/3) for x in max_precip_h4cn]

In [ ]:
# Uses Polymesh code to create polygons from mesh data
storm_names = [x for x in all_storms.keys() if x != 'Charley']
p_rprect_dfs = list(map(return_polymesh, storm_names, ['cam']*len(storm_names), ['PRECT_MAX']*len(storm_names)))
c_rprect_dfs = list(map(return_polymesh, storm_names, ['mpas']*len(storm_names), ['PRECT_MAX']*len(storm_names)))

In [ ]:
from metpy.calc import cape_cin, dewpoint_from_relative_humidity, parcel_profile
from metpy.units import units
# pressure
p = [1008., 1000., 950., 900., 850., 800., 750., 700., 650., 600.,
     550., 500., 450., 400., 350., 300., 250., 200.,
     175., 150., 125., 100., 80., 70., 60., 50.,
     40., 30., 25., 20.] * units.hPa
# temperature
T = [29.3, 28.1, 23.5, 20.9, 18.4, 15.9, 13.1, 10.1, 6.7, 3.1,
     -0.5, -4.5, -9.0, -14.8, -21.5, -29.7, -40.0, -52.4,
     -59.2, -66.5, -74.1, -78.5, -76.0, -71.6, -66.7, -61.3,
     -56.3, -51.7, -50.7, -47.5] * units.degC
# relative humidity
rh = [.85, .65, .36, .39, .82, .72, .75, .86, .65, .22, .52,
      .66, .64, .20, .05, .75, .76, .45, .25, .48, .76, .88,
      .56, .88, .39, .67, .15, .04, .94, .35] * units.dimensionless
# calculate dewpoint
Td = dewpoint_from_relative_humidity(T, rh)
# compture parcel temperature
prof = parcel_profile(p, T[0], Td[0]).to('degC')
# calculate surface based CAPE/CIN
cape_cin(p, T, Td, prof)[0]

In [ ]:
all_storms['storm_0236']['h3pn_ds']

In [ ]:
all_storms['storm_0236']['h3cn_ds']

In [ ]:
import datashader as ds
from holoviews import opts

def plot_precip(base_df):
    # Defines defaults
    w = 1000
    h = 700
    
    proj = ccrs.PlateCarree()
    lon_range, lat_range = basin_bboxes('florida')
    x_range, y_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
    x_range = tuple(x_range)
    y_range = tuple(y_range)
    
    plot_opts = dict(width=w, height=h, xaxis=None, yaxis=None)
    
    lw = 3.0
    coastline_opts = dict(scale='10m', line_color='#000000', line_width=lw, width=w, height=h)
    states_opts = dict(scale='50m', line_color='#000000', line_width=lw, width=w, height=h, fill_color='none')
    
    base_cmap = ['#ffffff', '#ffffff', "#4bd2f7", "#699fd0", "#3c4bac", 
                 "#3cf74b", "#3cb447", "#3c8743", "#1f4723", "#f7f73c"]
    base_levs = CustomColorbars('nws_precip').out_levels[:11]
    base_cmin = base_levs[0]
    base_cmax = base_levs[-1]
    base_cmap_opts = dict(cmap=base_cmap, clim=(base_cmin, base_cmax), colorbar=False)
    base_plot_opts = dict(**plot_opts, **base_cmap_opts, alpha=0.75, color_levels=base_levs)
    
    coastline_layer = gf.coastline(projection=proj).opts(**coastline_opts, xlim=x_range, ylim=y_range)
    states_layer = gf.states(projection=proj).opts(**states_opts, xlim=x_range, ylim=y_range)

    # Sets up basemap
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)

    # Adds base layer polygons
    agg_base = cvs.polygons(base_df, geometry='geometry', agg=ds.mean('faces'))

    # Rasterizes layer
    hv_img_base = hv.element.raster.Image(agg_base).opts(**base_plot_opts)

    # Adds geographic layers
    hv_img_base = hv_img_base * coastline_layer * states_layer
    
    return hv_img_base

In [ ]:
cam_conv_plots = [plot_precip(df) for df in p_rprect_dfs]

In [ ]:
mpas_conv_plots = [plot_precip(df) for df in c_rprect_dfs]

In [ ]:
c_rprect_dfs[0]

In [ ]:
[hv.save(fig, f'../figs/new_tracks/{storm}/cam_conv_rate_max.png', dpi=300, center=False) for fig, storm in zip(cam_conv_plots, storm_names)]

In [ ]:
[hv.save(fig, f'../figs/new_tracks/{storm}/mpas_conv_rate_max.png', dpi=300, center=False) for fig, storm in zip(mpas_conv_plots, storm_names)]